# 03 Flow Diagnostics IRF


このノートブックは `config/config.example.yaml` を読み込み、synthetic データで手法を単体検証します。

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "config" / "config.example.yaml").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import load_config
from src.utils.calendar import month_end_dates, quarter_end_dates
from src.utils.io import save_parquet, save_csv, save_figure
from src.utils.validation import assert_weights_sum_to_one, assert_no_lookahead

cfg = load_config(PROJECT_ROOT / "config" / "config.example.yaml")
np.random.seed(cfg["project"]["seed"])

for rel in ["outputs/logs", "outputs/signals", "outputs/weights", "outputs/diagnostics", "outputs/figures"]:
    (PROJECT_ROOT / rel).mkdir(parents=True, exist_ok=True)

def make_synthetic(cfg):
    dates = pd.bdate_range(cfg["data"]["date_start"], cfg["data"]["date_end"])
    themes = cfg["data"]["assets"]["theme_ids"]
    n, m = len(dates), len(themes)

    factors = np.random.normal(0.0, 0.004, size=(n, 3))
    loadings = np.random.uniform(-1.0, 1.0, size=(m, 3))
    eps = np.random.normal(0.0, 0.01, size=(n, m))
    returns = pd.DataFrame(factors @ loadings.T + eps, index=dates, columns=themes)
    prices = 100.0 * (1.0 + returns).cumprod()

    flow_common = np.random.normal(0.0, 1.0, size=n)
    flow_noise = np.random.normal(0.0, 0.6, size=(n, m))
    flow = pd.DataFrame(flow_common[:, None] + flow_noise, index=dates, columns=themes)

    theme_scores = flow.rolling(21, min_periods=5).mean() + np.random.normal(0.0, 0.2, size=(n, m))
    stale_cols = themes[2::2]
    if len(stale_cols) > 0:
        theme_scores.loc[dates[-120:], stale_cols] = np.nan

    if cfg["calendar"]["rebalance"] == "Q":
        rebalance_dates = quarter_end_dates(dates)
    else:
        rebalance_dates = month_end_dates(dates)

    return dates, themes, returns, prices, flow, theme_scores, rebalance_dates


In [ ]:
from src.signals.flow_diagnostics import compute_flow_pressure_score, local_projection_irf

dates, themes, returns, prices, flow, theme_scores, rebalance_dates = make_synthetic(cfg)

flow_window = cfg["flow_diagnostics"]["flow_window_days"]
pressure = pd.DataFrame({
    c: compute_flow_pressure_score(flow[c], window=flow_window, z=True) for c in themes
}, index=dates)
shock = pressure.mean(axis=1)
irf = local_projection_irf(
    returns=returns[themes[0]],
    shock=shock,
    horizons=range(1, cfg["flow_diagnostics"]["irf_horizons"] + 1),
    nw_lags=cfg["flow_diagnostics"]["nw_lags"],
)

assert pressure.shape == flow.shape
assert set(["h", "beta", "tstat", "nobs"]).issubset(irf.columns)

save_parquet(pressure, PROJECT_ROOT / "outputs/signals/03_flow_pressure_score.parquet")
save_csv(irf, PROJECT_ROOT / "outputs/diagnostics/03_flow_irf.csv")

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(irf["h"], irf["beta"], marker="o")
ax.axhline(0.0, color="black", linewidth=1.0)
ax.set_title("03 Local Projection IRF")
ax.set_xlabel("horizon")
ax.set_ylabel("beta")
save_figure(fig, PROJECT_ROOT / "outputs/figures/03_flow_irf.png")
plt.close(fig)

print(irf.head())


In [ ]:
required = [
    "outputs/signals/03_flow_pressure_score.parquet",
    "outputs/diagnostics/03_flow_irf.csv",
]
status = "OK" if all((PROJECT_ROOT / p).exists() for p in required) else "NG"
checks = ", ".join([f"{p}:{(PROJECT_ROOT / p).exists()}" for p in required])
line = f"Notebook=03_flow_diagnostics_irf.ipynb, {status}, 生成ファイル確認={checks}\n"
with (PROJECT_ROOT / "outputs/logs/qa_checklist.txt").open("a", encoding="utf-8") as f:
    f.write(line)
print(line)
